# 전이학습 (Transfer Learning)
* 이미 학습이 완료된 모델을 관련된 다른 도메인의 task에 대해 이용하는 방법
* 전이학습이 제대호 작용하는 배경으로는 source domain(기존의 도메인)과 관련된 전이할 곳의 target domain(목표 도메인)간에 낮은 레벨로 공통된 특징이 존재하고 있다는 가정이 있다.


In [34]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
model = VGG16()
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.preprocessing.image import load_img

## VGG16이 기본으로 제공하는 모델 이용

In [64]:
# 모델 확인하기
# 입력 크기: 224, 224, 3
# 출력 크기: 1000 (총 1000가지의 분류)
# 출력층 활성화 함수: SoftMax
model = VGG16()
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [65]:
model.get_config()

{'name': 'vgg16',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 224, 224, 3),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_20'},
   'name': 'input_20',
   'inbound_nodes': []},
  {'class_name': 'Conv2D',
   'config': {'name': 'block1_conv1',
    'trainable': True,
    'dtype': 'float32',
    'filters': 64,
    'kernel_size': (3, 3),
    'strides': (1, 1),
    'padding': 'same',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'groups': 1,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'name': 'block1_conv1',
   'inbound_nodes': [[['input_20', 0, 0, {}]]]},
  {'class_name': 'Conv2D',

In [66]:
# 내가 가지고 있는 이미지 데이터를 불러오기
image = load_img('bag_0.jpg', target_size=(224,224))

In [67]:
# 이미지 데이터를 배열 형태로 변경
image = img_to_array(image)

In [68]:
# 데이터의 배열을 (1, 224, 224, 3)으로 변경
image= image.reshape((1,image.shape[0], image.shape[1], image.shape[2]))
image.shape

(1, 224, 224, 3)

In [69]:
# vgg16이 제공하는 입력값 전처리 모듈을 활용
image = preprocess_input(image)

In [70]:
yhat = model.predict(image)

1/1 [==============================] - 0s 98ms/step


In [71]:
label = decode_predictions(yhat)

In [72]:
label

[[('n02769748', 'backpack', 0.94986606),
  ('n03709823', 'mailbag', 0.04527622),
  ('n02916936', 'bulletproof_vest', 0.0009227807),
  ('n03649909', 'lawn_mower', 0.00058785937),
  ('n04026417', 'purse', 0.00055419595)]]

In [73]:
label = label[0][0]
print('%s (%.2f%%)' % (label[1],label[2]*100))

backpack (94.99%)


## VGG16모델을 커스텀하여 사용

커스텀 모델로 설정하면 block5_pool 이후의 출력층이 없는 것을 확인할 수 있다.  
이후의 모델은 내가 원하는 출력층을 만드는 것이 가능하다.  
단, 커스텀 모델로 설정시 반드시 입력층 크기를 설정해둬야 한다.

In [75]:
conv_base = VGG16(include_top=False, input_shape=(224, 224, 3))
conv_base.trainable=False
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [57]:
# 출력 모델 생성하기
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [58]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_21 (Dense)            (None, 256)               6422784   
                                                                 
 dense_22 (Dense)            (None, 1)                 257       
                                                                 
Total params: 21,137,729
Trainable params: 6,423,041
Non-trainable params: 14,714,688
_________________________________________________________________


In [59]:
yhat2 = model.predict(image)

1/1 [==============================] - 0s 86ms/step


In [60]:
yhat2

array([[1.]], dtype=float32)

# Layer동결
* weights: 레이어의 모든 가중치 변수 목록
* trainable_weights: 훈련 중 손실을 최소화 하기 위해 엄데이트 되어야 하는 목록
* non_trainable_weights: 훈련되지 않은 것들의 목록, 일반적으로 순방향 전달 중에 모델에 의해 업데이트 된다.

In [35]:
# 예제 코드 - 인반적인 레이어의 모든 가중치는 훈련이 가능함
layer = keras.layers.Dense(3)
layer.build((None, 4))  # Create the weights

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

weights: 2
trainable_weights: 2
non_trainable_weights: 0


In [36]:
# 예제 코드2 - 훈련할 수 없는 가중치가 있는 유일한 내장 레이어
# BatchNormalization층에는 총 4개의 가중치가 있다.
# 4개중 2개의 가중치가 훈련이 가능하며 나머지 2개는 훈련이 불가능하다.
layer = keras.layers.BatchNormalization()
layer.build((None, 4))  # Create the weights

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

weights: 4
trainable_weights: 2
non_trainable_weights: 2


In [37]:
# 예제 코드3 - 설정을 통해 가중치를 훈련 불가 가중치로 변경
layer = keras.layers.Dense(3)
layer.build((None, 4))  # Create the weights
layer.trainable = False  # 파라미터를 학습시키지 않고 기존의 값들을 고정시킴

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

weights: 2
trainable_weights: 0
non_trainable_weights: 2


In [43]:
# 2개의 레이어를 갖는 모델 생성
layer1 = keras.layers.Dense(3, activation="relu")
layer2 = keras.layers.Dense(3, activation="sigmoid")
model = keras.Sequential([keras.Input(shape=(3,)), layer1, layer2])

# 1번층의 가중치를 동결시킴
layer1.trainable = False

# 가중치가 훈련되지 않았음을 확인하기 위해 현재 가중치를 저장
initial_layer1_weights_values = layer1.get_weights()

# 모델 훈련
model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

# 훈련중 1번층의 가중치가 변경되지 않았음을 확인
final_layer1_weights_values = layer1.get_weights()
# np.testing.assert_allclose는 두 개체가 
# 원하는 허용 오차까지 같지 않으면 AssertionError를 발생시킨다.
# 현재 이 코드의 경우 Error가 발생하지 않으므로 가중치가 업데이트 되지 않았음을 알 수 있다.
np.testing.assert_allclose(
    initial_layer1_weights_values[0], final_layer1_weights_values[0]
)
np.testing.assert_allclose(
    initial_layer1_weights_values[1], final_layer1_weights_values[1]
)

1/1 [==============================] - 0s 123ms/step - loss: 0.0962


## trainable 속성의 재귀 설정
* 모델 또는 하위 레이어가 있는 레이어에서 trainable - Flase를 설정하면  
  모든 하위 레이어도 훈련할 수 없게 된다

In [44]:
inner_model = keras.Sequential(
    [
        keras.Input(shape=(3,)),
        keras.layers.Dense(3, activation="relu"),
        keras.layers.Dense(3, activation="relu"),
    ]
)

model = keras.Sequential(
    [keras.Input(shape=(3,)), inner_model, keras.layers.Dense(3, activation="sigmoid"),]
)

model.trainable = False  # Freeze the outer model

assert inner_model.trainable == False  # All layers in `model` are now frozen
assert inner_model.layers[0].trainable == False  # `trainable` is propagated recursively